In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import spacy
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import numpy as np
import nltk
import random
from langdetect import detect
from pathlib import Path
from textblob import TextBlob
from spacy.util import minibatch, compounding
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [5]:
def lemmatise_text(text):
    tweet_nlp = nlp(text)
    l = [token.lemma_ for token in tweet_nlp ]
    return (" ".join(l)).replace(" \n","")

def stem_text(text):
    words = word_tokenize(text)
    l = [stemmer.stem(token) for token in words]
    return " ".join(l)


In [8]:
nlp = spacy.load("C:/Users/Benco/Anaconda3/Lib/site-packages/fr_core_news_sm/fr_core_news_sm-2.2.5")

In [9]:
stemmer = SnowballStemmer('french')

In [10]:
train = pd.read_csv("train_data.csv")
train = train[["question" , "reponse"]]
train["lemmas_rep"] = train["reponse"].apply(lemmatise_text)
train["lemmas_quest"] = train["question"].apply(lemmatise_text)
train["stem_rep"] = train["reponse"].apply(stem_text)
train["stem_quest"] = train["question"].apply(stem_text)

In [11]:
train.to_csv("training_data.csv" , index = False)

In [12]:
train = pd.read_csv("training_data.csv")

In [14]:
train

,question,reponse,lemmas_rep,lemmas_quest,stem_rep,stem_quest
0,Quels sont les forfaits vacances disponibles ?,Nos forfaits vacances répondent à tous vos bes...,notre forfait vacance répondre à tout votre be...,quel être le forfait vacance disponible ?,nos forfait vacanc répondent à tous vos besoin...,quel sont le forfait vacanc disponibl ?
1,Comment supprimer les cookies Flash ?,Voici comment modifier les paramètres Flash. S...,voici comment modifier le paramètre flash . so...,comment supprimer le cookie flash ?,voic comment modifi le parametr flash . souven...,comment supprim le cook flash ?
2,Quels types de restaurants trouve t-on dans le...,Disneyland Paris dispose de plus de 65 restaur...,Disneyland Paris dispose de plus de 65 restaur...,quel type de restaurant trouve t - on dans le ...,disneyland paris dispos de plus de 65 restaur ...,quel typ de restaur trouv t-on dan le parc dis...
3,Quels concerts y a-t-il actuellement au Disney...,Découvrez les superbes concerts qui se déroule...,Découvrez le superbe concert qui se dérouler b...,quel concerts y avoir t il actuellement au Dis...,découvr le superb concert qui se déroul bientô...,quel concert y a-t-il actuel au disney villag ?
4,Comment connaître la disponibilité des chambre...,Vous pouvez consulter la disponibilité des cha...,vous pouvoir consulter le disponibilité un cha...,comment connaître le disponibilité un chambre ...,vous pouv consult la disponibil de chambr d'hô...,comment connaîtr la disponibil de chambr avant...
...,...,...,...,...,...,...
139,Où se trouve le centre Accueil Animaux ?,Le centre Accueil Animaux se trouve à proximit...,le centre Accueil Animaux se trouve à proximit...,où se trouve le centre Accueil Animaux ?,le centr accueil animal se trouv à proxim du p...,où se trouv le centr accueil animal ?
140,"Je souffre d'intolérances alimentaires, où pui...",Pas d'inquiétude. Les restaurants Disney sont ...,pas de inquiétude . le restaurant Disney être ...,"je souffrir de intolérance alimentaire , où pu...",pas d'inquiétud . le restaur disney sont heure...,"je souffr d'intoler alimentair , où puis-j mang ?"
141,Est-il possible d’acheter un PhotoPass+ Annuel...,"Oui, selon les conditions tarifaires en vigueu...","oui , selon le condition tarifaire en vigueur ...",est - il possible d’ acheter un photopass+ ann...,"oui , selon le condit tarifair en vigueur au m...",est-il possibl d ’ achet un photopass+ annuel ...
142,Y a-t-il des poussettes dans les Parcs Disney ?,Oui. Il est possible de louer des poussettes a...,oui . il être possible de louer un poussette a...,y avoir t il un poussette dans le parc Disney ?,oui . il est possibl de lou de pousset au poin...,y a-t-il de pousset dan le parc disney ?


In [ ]:
def model_build(X_train , stop_words):
    vectorizer = TfidfVectorizer(lowercase=True, stop_words = stop_words,
                            ngram_range=(1, 1),
                            use_idf=True, smooth_idf=True, # idf lissé
                            sublinear_tf=False, norm='l2')
    vectorizer.fit(pd.concat([X_train["stem_quest"] , X_train["stem_rep"]]))
    
    
    

In [15]:
sw = nltk.corpus.stopwords.words('french')
sw += ['être', 'avoir']

In [37]:
vectorizer = TfidfVectorizer(lowercase=True, stop_words = sw,
                            ngram_range=(1, 1),
                            use_idf=True, smooth_idf=True, # idf lissé
                            sublinear_tf=False, norm='l2')

In [39]:
X_train_vectorized_tfidf = vectorizer.fit_transform(pd.concat([train["stem_quest"] , train["stem_rep"]]))# compléter ici

In [55]:
query = [stem_text("Quelles est la couleur de la pate?")]
query_vector = vectorizer.transform(query)
pd.DataFrame(query_vector.toarray(), columns=vectorizer.get_feature_names())

,000,01,02,024,03,078,10,100,100g,10h,...,établ,étap,état,étendu,étroit,évad,éven,évit,ête,êtr
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
query_corpus_sim = np.squeeze(cosine_similarity(X_train_vectorized_tfidf, query_vector)) ## on enlève leur format qui n'est pas bon --> vecteur
np.max(query_corpus_sim)

0.4518993097307528

In [57]:
id_best_doc = np.argsort(query_corpus_sim)[-1] ## ordre croissant
train["reponse"][id_best_doc]

'Consultez les coordonnées de Disneyland Paris.'